In [54]:
import pandas as pd
import numpy as np
import io # Usado para cargar datos de ejemplo desde un string
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocesamiento y Pipelines
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Modelos
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# Métricas
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score, roc_auc_score , precision_score, recall_score

# Configuraciones
sns.set_style('whitegrid') # Estilo de gráficos
pd.set_option('display.max_columns', None) #mostrar todas las columnas al imprimir dataframes

In [4]:
#Leer archivo CSV
data= pd.read_csv("../Datos/e-shop clothing 2008.csv", sep=";")  # Cargar datos desde un archivo CSV

In [5]:
data

,year,month,day,order,country,session ID,page 1 (main category),page 2 (clothing model),colour,location,model photography,price,price 2,page
0,2008,4,1,1,29,1,1,A13,1,5,1,28,2,1
1,2008,4,1,2,29,1,1,A16,1,6,1,33,2,1
2,2008,4,1,3,29,1,2,B4,10,2,1,52,1,1
3,2008,4,1,4,29,1,2,B17,6,6,2,38,2,1
4,2008,4,1,5,29,1,2,B8,4,3,2,52,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165469,2008,8,13,1,29,24024,2,B10,2,4,1,67,1,1
165470,2008,8,13,1,9,24025,1,A11,3,4,1,62,1,1
165471,2008,8,13,1,34,24026,1,A2,3,1,1,43,2,1
165472,2008,8,13,2,34,24026,3,C2,12,1,1,43,1,1


In [23]:
df = data.copy() # Copia del df principal
df['Visited_Sale_Page'] = np.where(df['page 1 (main category)'] == 4, 1, 0) #Creación Flag 
df_filter = df.copy() #Copia del df para filtrar
df_filter = df_filter[df_filter['Visited_Sale_Page'] != 1] #Filtro de filas donde se entro a la pagina de ofertas

In [24]:
#Crear columnas agregadas (features) por 'session ID'

df_agg = df_filter.groupby('session ID').agg(
    total_clicks=('order', 'count'),
    max_price_seen=('price', 'max'),
    avg_price_seen=('price', 'mean'),
    countries=('country', pd.Series.nunique),
    main_category_mode=('page 1 (main category)', lambda x: x.mode()[0]),
    distinct_products=('page 2 (clothing model)', pd.Series.nunique)
).reset_index()

In [22]:
df_agg

,session ID,total_clicks,max_price_seen,avg_price_seen,countries,main_category_mode,distinct_products
0,1,7,57,41.857143,1,2,7
1,2,7,67,53.000000,1,2,5
2,3,5,48,42.000000,1,3,5
3,4,4,62,45.250000,1,1,4
4,5,1,57,57.000000,1,3,1
...,...,...,...,...,...,...,...
22279,24022,3,33,29.666667,1,1,3
22280,24023,4,38,35.500000,1,1,4
22281,24024,1,67,67.000000,1,2,1
22282,24025,1,62,62.000000,1,1,1


In [ ]:
#Crear df con la variable objetivo por sesión
df_target = df.groupby('session ID')['Visited_Sale_Page'].max().reset_index()

In [34]:
#Hacer merge de features con variable objetivo
df_sesiones = pd.merge(df_agg, df_target, on='session ID')

In [35]:
#Proporción de clases en la variable objetivo
df_sesiones['Visited_Sale_Page'].value_counts(normalize=True)

Visited_Sale_Page
0    0.665679
1    0.334321
Name: proportion, dtype: float64

In [37]:
#Eliminar columna 'session ID' ya que no es necesaria para el modelado
df_sesiones.drop('session ID', axis=1, inplace=True)

In [38]:
df_sesiones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22284 entries, 0 to 22283
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   total_clicks        22284 non-null  int64  
 1   max_price_seen      22284 non-null  int64  
 2   avg_price_seen      22284 non-null  float64
 3   countries           22284 non-null  int64  
 4   main_category_mode  22284 non-null  int64  
 5   distinct_products   22284 non-null  int64  
 6   Visited_Sale_Page   22284 non-null  int64  
dtypes: float64(1), int64(6)
memory usage: 1.2 MB


## Pipiline de procesamiento

In [42]:
# Definir las columnas
numeric_features = ['total_clicks', 'max_price_seen', 'avg_price_seen']

# Seleccionar automáticamente todas las columnas 'object' como categóricas

categorical_features = df_sesiones.select_dtypes(include=['object', 'category']).columns.tolist()

for col in ['countries', 'main_category_mode', 'distinct_products']:
    if col not in categorical_features:
        categorical_features.append(col)

# Asegurarnos de que las numéricas no estén en la lista de categóricas
categorical_features = [col for col in categorical_features if col not in numeric_features and col != 'Visited_Sale_Page']

print(f"Features Numéricas: {numeric_features}")
print(f"Features Categóricas: {categorical_features}")

# Crear el transformador para variables numéricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), # Imputar NAs con la mediana
    ('scaler', StandardScaler())                 # Escalar
])

# Crear el transformador para variables categóricas
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # Imputar NAs con la moda
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))   # Aplicar One-Hot Encoding
])

# Combinar los transformadores usando ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough' # Dejar pasar cualquier columna no especificada (aunque no debería haber)
)

print("\nPreprocesador creado exitosamente!")


Features Numéricas: ['total_clicks', 'max_price_seen', 'avg_price_seen']
Features Categóricas: ['countries', 'main_category_mode', 'distinct_products']

Preprocesador creado exitosamente!


## División de los datos de entrenamiento y de testeo

In [44]:
X = df_sesiones.drop('Visited_Sale_Page', axis=1)
y = df_sesiones['Visited_Sale_Page']

# Usamos un random_state fijo (ej. 42) para que nuestros resultados sean reproducibles
# Usamos 'stratify=y' para asegurar que la proporción de Churn sea la misma en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Tamaño de Train: {X_train.shape}")
print(f"Tamaño de Test: {X_test.shape}")

Tamaño de Train: (17827, 6)
Tamaño de Test: (4457, 6)


### Primer modelo: Regresión logistica

In [55]:
# Crear pipeline con preprocesador y regresión logística
clf = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced'))
])

# Entrenar el pipeline
print("Entrenando el Pipeline de Regresión Logística...")
clf.fit(X_train, y_train)
print("Entrenamiento completado.")

# Evaluar el modelo
y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)[:, 1]

print("\n--- Reporte de Clasificación (Regresión Logística) ---")
print(classification_report(y_test, y_pred))

# Guardar métricas
logreg_accuracy = accuracy_score(y_test, y_pred)
logreg_f1 = f1_score(y_test, y_pred)
logreg_auc = roc_auc_score(y_test, y_prob)
logreg_precision = precision_score(y_test, y_pred, pos_label=1)
logreg_recall = recall_score(y_test, y_pred, pos_label=1)

Entrenando el Pipeline de Regresión Logística...
Entrenamiento completado.

--- Reporte de Clasificación (Regresión Logística) ---
              precision    recall  f1-score   support

           0       0.77      0.62      0.69      2967
           1       0.45      0.63      0.53      1490

    accuracy                           0.62      4457
   macro avg       0.61      0.63      0.61      4457
weighted avg       0.66      0.62      0.63      4457



### Segundo modelo: Arbol de decisión

In [56]:
# Crear pipeline con preprocesador y árbol de decisión
dt_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(random_state=42, class_weight='balanced'))
])

# Entrenar el pipeline
print("Entrenando el Pipeline de Árbol de Decisión...")
dt_pipeline.fit(X_train, y_train)
print("Entrenamiento completado.")

# Evaluar
y_pred_dt = dt_pipeline.predict(X_test)
# Para árboles, predict_proba también está disponible
y_prob_dt = dt_pipeline.predict_proba(X_test)[:, 1]

print("\n--- Reporte de Clasificación (Árbol de Decisión) ---")
print(classification_report(y_test, y_pred_dt))

# Guardar métricas
dt_accuracy = accuracy_score(y_test, y_pred_dt)
dt_f1 = f1_score(y_test, y_pred_dt)
dt_auc = roc_auc_score(y_test, y_prob_dt)
dt_precision = precision_score(y_test, y_pred_dt, pos_label=1)
dt_recall = recall_score(y_test, y_pred_dt, pos_label=1)

Entrenando el Pipeline de Árbol de Decisión...
Entrenamiento completado.

--- Reporte de Clasificación (Árbol de Decisión) ---
              precision    recall  f1-score   support

           0       0.71      0.74      0.72      2967
           1       0.43      0.40      0.42      1490

    accuracy                           0.63      4457
   macro avg       0.57      0.57      0.57      4457
weighted avg       0.62      0.63      0.62      4457



### Tercer modelo: Ramdon Forest

In [57]:
# Crear el Pipeline de Random Forest
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestClassifier(random_state=42, n_jobs=-1)) # n_jobs=-1 usa todos los cores
])

# Entrenar el pipeline
print("Entrenando el Pipeline de Random Forest...")
rf_pipeline.fit(X_train, y_train)
print("Entrenamiento completado.")

# Evaluar
y_pred_rf = rf_pipeline.predict(X_test)
y_prob_rf = rf_pipeline.predict_proba(X_test)[:, 1] # Probabilidad de la clase 1 (Churn)

print("\n--- Reporte de Clasificación (Random Forest) ---")
print(classification_report(y_test, y_pred_rf))

# Guardar métricas
rf_accuracy = accuracy_score(y_test, y_pred_rf)
rf_f1 = f1_score(y_test, y_pred_rf)
rf_auc = roc_auc_score(y_test, y_prob_rf)
rf_precision = precision_score(y_test, y_pred_rf, pos_label=1)
rf_recall = recall_score(y_test, y_pred_rf, pos_label=1)

Entrenando el Pipeline de Random Forest...
Entrenamiento completado.

--- Reporte de Clasificación (Random Forest) ---
              precision    recall  f1-score   support

           0       0.70      0.82      0.76      2967
           1       0.47      0.30      0.37      1490

    accuracy                           0.65      4457
   macro avg       0.58      0.56      0.56      4457
weighted avg       0.62      0.65      0.63      4457



### Cuarto modelo: XGBoost

In [ ]:
# Calcular scale_pos_weight para manejar el desbalanceo
scale_pos_weight = y_train.value_counts()[0] / y_train.value_counts()[1]
print(f"Scale Pos Weight: {scale_pos_weight:.2f}")

# Crear el Pipeline de XGBoost
# Nota: Añadimos 'use_label_encoder=False' y 'eval_metric='logloss'' 
# para evitar warnings comunes en versiones recientes de XGBoost.
xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', XGBClassifier(
        random_state=42, 
        use_label_encoder=False, 
        eval_metric='logloss', 
        n_jobs=-1,
        scale_pos_weight=scale_pos_weight # Ayudar con el desbalanceo
    ))
])

# Entrenar el pipeline
print("Entrenando el Pipeline de XGBoost...")
xgb_pipeline.fit(X_train, y_train)
print("Entrenamiento completado.")

# Evaluar
y_pred_xgb = xgb_pipeline.predict(X_test)
y_prob_xgb = xgb_pipeline.predict_proba(X_test)[:, 1]

print("\n--- Reporte de Clasificación (XGBoost) ---")
print(classification_report(y_test, y_pred_xgb))

# Guardar métricas
xgb_accuracy = accuracy_score(y_test, y_pred_xgb)
xgb_f1 = f1_score(y_test, y_pred_xgb)
xgb_auc = roc_auc_score(y_test, y_prob_xgb)
xgb_precision = precision_score(y_test, y_pred_xgb, pos_label=1)
xgb_recall = recall_score(y_test, y_pred_xgb, pos_label=1)

Scale Pos Weight: 1.99
Entrenando el Pipeline de XGBoost...


c:\Users\inter\AppData\Local\pypoetry\Cache\virtualenvs\arboles-decision-zrGSN9Ih-py3.13\Lib\site-packages\xgboost\training.py:199: UserWarning: [17:45:09] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Entrenamiento completado.

--- Reporte de Clasificación (XGBoost) ---
              precision    recall  f1-score   support

           0       0.76      0.64      0.69      2967
           1       0.45      0.59      0.51      1490

    accuracy                           0.63      4457
   macro avg       0.61      0.62      0.60      4457
weighted avg       0.66      0.63      0.63      4457



## Comparando los modelos

In [59]:
# Crear diccionario con métricas de cada modelo
metrics_dict = {
    'Modelo': ['Regresión Logística', 'Árbol de Decisión', 'Random Forest', 'XGBoost'],
    'Accuracy': [logreg_accuracy, dt_accuracy, rf_accuracy, xgb_accuracy],
    'F1 Score': [logreg_f1, dt_f1, rf_f1, xgb_f1],
    'ROC AUC': [logreg_auc, dt_auc, rf_auc, xgb_auc],
    'Precision': [logreg_precision, dt_precision, rf_precision, xgb_precision],
    'Recall': [logreg_recall, dt_recall, rf_recall, xgb_recall]
}

# Crear DataFrame
metrics_df = pd.DataFrame(metrics_dict)

print(metrics_df)

                Modelo  Accuracy  F1 Score   ROC AUC  Precision    Recall
0  Regresión Logística  0.622616  0.528323  0.670747   0.453757  0.632215
1    Árbol de Decisión  0.625084  0.417161  0.542570   0.434277  0.401342
2        Random Forest  0.650438  0.367695  0.606180   0.465092  0.304027
3              XGBoost  0.625084  0.513820  0.652762   0.453518  0.592617
